# MovieLens Data Preprocessing Notebook
- This notebook contains step-by-step approach taken to clean, modify and preprocess various data files.
- Some data is normalized and new tables are created to making data analysis easy.

In [1]:
#Imports
import pandas as pd
from datetime import datetime

#setting
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Movies file

This file contains 3 columns.
- **movieId**: Unique identifier for each movie
- **title**: Name of the movie
- **genres**: pipe delimeted text column specifying various genres a movie belongs to.

We will take a look at the structure of the table and find if there are any null values.
Then split the genres column into multiple columns and create a new table to normalize data.

In [2]:
#Read file
movies = pd.read_csv("movies.csv")
movies.info()
movies.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58098 entries, 0 to 58097
Data columns (total 3 columns):
movieId    58098 non-null int64
title      58098 non-null object
genres     58098 non-null object
dtypes: int64(1), object(2)
memory usage: 1.3+ MB


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [77]:
#check null values
movies.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

In [78]:
###split genres
genre_split = movies['genres'].str.split('|', expand=True)
movies_genres = pd.concat([movies.loc[:, movies.columns != 'genres'], genre_split], axis=1)

###Reshape
movies_genres = movies_genres.melt(id_vars=['movieId', 'title'], var_name='genre_no', value_name='genre')
movies_genres = movies_genres.drop(['title','genre_no'], axis=1)
movies_genres = movies_genres[movies_genres['genre'].notnull()]

movies_genres = movies_genres.sort_values('movieId').reset_index(drop=True)

In [79]:
movies_genres.head(10)

,movieId,genre
0,1,Adventure
1,1,Fantasy
2,1,Animation
3,1,Comedy
4,1,Children
5,2,Fantasy
6,2,Children
7,2,Adventure
8,3,Romance
9,3,Comedy


In [80]:
#drop genres from original movie file
movies = movies.drop('genres', axis=1)
movies.head(10)

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)
5,6,Heat (1995)
6,7,Sabrina (1995)
7,8,Tom and Huck (1995)
8,9,Sudden Death (1995)
9,10,GoldenEye (1995)


This is the modified version of movies file.

### Ratings file

Ratings file has ratings from each use for every movie they rated.

- In this file the timestamp is mentioned in seconds after epoch. We shall convert it to datatime stamp
- Aggregate average ratings and number of rantings merge them to movie file

In [18]:
#Read file
ratings = pd.read_csv("ratings.csv")
ratings.info()
ratings.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27753444 entries, 0 to 27753443
Data columns (total 4 columns):
userId       int64
movieId      int64
rating       float64
timestamp    int64
dtypes: float64(1), int64(3)
memory usage: 847.0 MB


,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


In [19]:
#convert timestamp
ratings['datetime'] = ratings['timestamp'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
ratings.head(10)

,userId,movieId,rating,timestamp,datetime
0,1,307,3.5,1256677221,2009-10-27 21:00:21
1,1,481,3.5,1256677456,2009-10-27 21:04:16
2,1,1091,1.5,1256677471,2009-10-27 21:04:31
3,1,1257,4.5,1256677460,2009-10-27 21:04:20
4,1,1449,4.5,1256677264,2009-10-27 21:01:04
5,1,1590,2.5,1256677236,2009-10-27 21:00:36
6,1,1591,1.5,1256677475,2009-10-27 21:04:35
7,1,2134,4.5,1256677464,2009-10-27 21:04:24
8,1,2478,4.0,1256677239,2009-10-27 21:00:39
9,1,2840,3.0,1256677500,2009-10-27 21:05:00


In [27]:
#drop timestamp column
ratings = ratings.drop('timestamp', axis=1)

In [22]:
#number of rating and average rating group by movies
ratings_agg = ratings[['movieId','rating']].groupby('movieId').agg(['count', 'mean'])

In [23]:
#drop multiindex
ratings_agg.columns = ratings_agg.columns.droplevel()

In [24]:
ratings_agg.head()

,count,mean
movieId,,
1,68469,3.886649
2,27143,3.246583
3,15585,3.173981
4,2989,2.874540
5,15474,3.077291


In [25]:
#add above values to movies file
movies = movies.join(ratings_agg, on='movieId')

In [26]:
movies.head(10)

,movieId,title,count,mean
0,1,Toy Story (1995),68469.0,3.886649
1,2,Jumanji (1995),27143.0,3.246583
2,3,Grumpier Old Men (1995),15585.0,3.173981
3,4,Waiting to Exhale (1995),2989.0,2.874540
4,5,Father of the Bride Part II (1995),15474.0,3.077291
5,6,Heat (1995),28683.0,3.844211
6,7,Sabrina (1995),15301.0,3.371348
7,8,Tom and Huck (1995),1539.0,3.122482
8,9,Sudden Death (1995),4449.0,3.007530
9,10,GoldenEye (1995),33086.0,3.431633


### ML Links file

This file contains movie ID's mapped from MovieLens, IMDB and TheMovieDB.

Our goal is to get movie ratings from both IMDB and TMDB. Both websites provide API's.
- We shall use IMDB provided downloadable movie ratings file.
- For TMDB we shall use API to get the relavent data.

In [111]:
#read file
ml_links = pd.read_csv('links.csv')
ml_links.head(10)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
5,6,113277,949.0
6,7,114319,11860.0
7,8,112302,45325.0
8,9,114576,9091.0
9,10,113189,710.0


### IMDB Rating
Downloaded from https://www.imdb.com/interfaces/

In [112]:
#read file
imdb_ratings = pd.read_csv('data.tsv', sep='\t')

In [113]:
#convert the string to match ID's from movielens movie file
imdb_ratings['imdbId'] = imdb_ratings['tconst'].str.strip('tt').astype('int')
imdb_ratings = imdb_ratings.drop('tconst', axis=1)

In [114]:
imdb_ratings.head()

,averageRating,numVotes,imdbId
0,5.7,1674,1
1,6.1,208,2
2,6.5,1400,3
3,6.2,123,4
4,6.2,2193,5


In [115]:
#join with movies file
ml_links = ml_links.merge(imdb_ratings, how='left', on='imdbId')

In [117]:
ml_links = ml_links.rename(columns={'averageRating':'averageRating_imdb', 'numVotes':'numVotes_imdb'})
ml_links.head()

,movieId,imdbId,tmdbId,averageRating_imdb,numVotes_imdb
0,1,114709,862.0,8.3,885001.0
1,2,113497,8844.0,7.0,308163.0
2,3,113228,15602.0,6.7,24219.0
3,4,114885,31357.0,6.0,9535.0
4,5,113041,11862.0,6.1,34441.0


### TMDB Ratings

{'adult': False, 'backdrop_path': '/3Rfvhy1Nl6sSGJwyjb0QiZzZYlB.jpg', 'belongs_to_collection': {'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}, 'budget': 30000000, 'genres': [{'id': 16, 'name': 'Animation'}, {'id': 12, 'name': 'Adventure'}, {'id': 10751, 'name': 'Family'}, {'id': 35, 'name': 'Comedy'}], 'homepage': 'http://toystory.disney.com/toy-story', 'id': 862, 'imdb_id': 'tt0114709', 'original_language': 'en', 'original_title': 'Toy Story', 'overview': "Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.", 'popularity': 125.942, 'poster_path': '/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg', 'production_companies': [{'id': 3, 'logo_path': '/1TjvG